<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Linear Regression
    </div>
<br><br>
<font size=4>
   **Name**: Bahar Dibaeinia <br>
   **Student ID**: 99105442 <br> <br>

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 1: Linear Regression Model (40 + 30 optional points)
According to <a href="https://github.com/asharifiz/Introduction_to_Machine_Learning/blob/main/Jupyter_Notebooks/Chapter_02_Classical_Models/Linear%20regression.ipynb"><font face="Roboto">Linear Regression Notebook</font></a>, train a linear regression model on an arbitrary dataset. Explain your chosen dataset and split your data into train and test sets, then predict values for the test set using your trained model. Try to find the best hyperparameters for your model. (Using Lasso Regression, Ridge Regression or Elastic Net and comparing them will have extra optional points)
<br> Explain each step of your workflow.

In [52]:

import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('housing.csv')
df.drop(['ocean_proximity'], axis=1, inplace=True) #removing last column
df = df.sample(frac = 1) #shuffle
df=df.dropna() #removing rows with null features
df_test=df.iloc[:int(len(df)*0.2),:] #test dataset
df=df.iloc[int(len(df)*0.2):,:]  #df = train dataset

means=df.iloc[:,:-1].mean() #normalization
stds=df.iloc[:,:-1].std()
df.iloc[:,:-1]=(df.iloc[:,:-1]-means)/stds #normalization
df_test.iloc[:,:-1]=(df_test.iloc[:,:-1]-means)/stds
df
#TODO

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
5427,0.577114,-0.757571,0.427900,0.168479,0.963343,1.066826,1.175414,-0.855081,450000.0
14703,1.251273,-1.329041,-0.447186,0.032554,-0.299650,-0.289279,-0.227591,0.681130,195100.0
10826,0.821809,-0.930886,-0.128973,-0.615035,-0.309146,-0.316720,-0.319205,-0.588518,275000.0
9792,-1.145735,0.427526,-0.685845,3.398643,2.924305,1.471356,2.939641,0.666699,387500.0
8295,0.716940,-0.884044,0.825666,-0.289181,0.232137,-0.490217,0.180746,0.106646,400000.0
...,...,...,...,...,...,...,...,...,...
5936,0.821809,-0.696677,-1.163165,3.195900,3.486954,2.923062,3.688259,0.085626,156800.0
11736,-0.741240,1.518940,-0.924505,2.292021,1.775267,1.482864,1.908327,0.217600,184300.0
1124,-1.000916,1.931148,-0.844952,-0.435175,-0.484825,-0.608832,-0.471023,-1.091787,103800.0
4340,0.642033,-0.729466,-0.447186,-0.349134,0.951473,1.216423,0.942452,-1.059368,212500.0


In [54]:
def mse_loss(pred, real):
    diff = pred - real
    return np.sum(diff * diff) / diff.size

def final_loss(pred, real, regularization, W, ratio, alpha):
    loss = mse_loss(pred, real)
    if regularization == 1:
        loss += alpha * (W.T@W).reshape(1,).item()

    elif regularization == 2:
        loss += alpha * np.sum(np.absolute(W))

    elif regularization == 3:
        loss += alpha * ratio * np.sum(np.absolute(W))
        loss += 0.5 * alpha * (1 - ratio) * (W.T@W).reshape(1,).item()

    return loss

def lr_predict(X, W):
    return np.dot(X, W)

In [55]:
X = df.iloc[:, :-1].to_numpy()
Y = df.iloc[:, -1:].to_numpy()
X_t = df_test.iloc[:, :-1].to_numpy()
Y_t = df_test.iloc[:, -1:].to_numpy()

In [59]:
import numpy as np
X = np.append(np.ones((len(X), 1)), X , axis=1) #for adding b we append a col of 1s
X_t = np.append(np.ones((len(X_t), 1)), X_t , axis=1)
print(X.shape)
print(X_t.shape)


(16347, 10)
(4086, 10)


In [60]:
def calc_gradient(X, error, regularization, W, ratio, alpha):
    gradient = np.dot(X.T, error)
    if regularization == 1:
        gradient += 2*W*alpha
   
    elif regularization == 2:
        gradient+=alpha*np.sign(W)
    
    elif regularization == 3:
        gradient+= ratio*alpha*np.sign(W)
        gradient += 0.5 * (1-ratio) * alpha * np.sign(W)
    
    return gradient
def update_weights(W, lr, gradient):
    new_weights = W - lr * gradient
    return new_weights


In [58]:
def train_model_lr_gd(X, Y, W, n_epoches, lr, regularization, ratio, alpha):
    #1:ridge
    #2:lasso
    #3:elastic
    losses = []
    for i in range(n_epoches):
        # generating the predictions
        predictions = lr_predict(X, W)
        error = predictions - Y
        # calculating the loss
        loss = final_loss(predictions, Y, regularization, W, ratio, alpha)
        # adding the loss to our loss list
        losses.append(loss)
        # calculating gradients
        gradient = calc_gradient(X, error, regularization, W, ratio, alpha) / Y.size
        # updating weights and biases
        W = update_weights(W, lr, gradient)
    return W, losses

In [64]:
W = np.random.rand(X.shape[1], 1)
lr = 1e-3
n_epoches = 10000
W2, losses = train_model_lr_gd(X, Y, W, n_epoches, lr, 1,0,0.3)
print('loss on train',losses[-1])
print('loss on test',final_loss(lr_predict(X_t, W2),Y_t,1,W2,0,0.3))

loss on train 15459583896.431328
loss on test 15431785669.576859


In [ ]:
W1 = np.random.rand(X.shape[1], 1)
losses_ratio = []
for i in range(11):

    W,losses = train_model_lr_gd(X, Y, W1, n_epoches, lr, 3,i*0.1,0.1)
    losses_ratio.append(losses[-1])

x = [i * 0.1 for i in range(11)]
y = losses_ratio
plt.plot(x, y)
plt.xlabel('ratio')
plt.ylabel('loss')
plt.show()

In [ ]:
!pip install matplotlib